In [3]:
import numpy as np
import scipy as sp
import random
import cmath
import math

# Define our buildDataSet function here!

In [4]:
def buildDataSet(max_amplitude,min_sparsity,max_sparsity,vector_size,data_set_size):
    sparse_data = np.zeros((vector_size,data_set_size)) # Initialize the sparse_data matrix

    # Iterate over the columns of the sparse_data matrix to define the data samples
    for i in range(data_set_size):
        sparsity = random.randint(min_sparsity,max_sparsity)
        indices = random.sample(range(vector_size),sparsity)
        amps = random.sample(range(max_amplitude),sparsity)
        sparse_data[indices,i] = amps
    
    # Define the DFT matrix and multiply our spare_data vectors with it to find dense data
    DFT = sp.linalg.dft(vector_size)/np.sqrt(vector_size)
    dense_data = DFT@sparse_data
    return dense_data,sparse_data

## Build the dataset

In [5]:
max_amplitude = 10
min_sparsity = 3
max_sparsity = 5
vector_size = 100
data_set_size = 1000
dense_data, sparse_data = buildDataSet(max_amplitude,min_sparsity,max_sparsity,vector_size,data_set_size)    

## Test the dataset

In [11]:
DFT = sp.linalg.dft(vector_size)/np.sqrt(vector_size)
iDFT = DFT.conj().T

iDFT@dense_data

array([[ 2.99610345e-16-1.84663568e-14j,  4.18125245e-15+2.93396784e-14j,
        -1.45951537e-15+2.79218236e-15j, ...,
         1.06855842e-14+6.18150753e-15j,  1.18998974e-14-4.44133976e-15j,
         5.66867945e-15+3.52969263e-15j],
       [-8.41028613e-15-2.31454568e-14j,  1.98885988e-14-1.13984708e-14j,
         2.83982701e-15+4.11576218e-16j, ...,
         2.17603713e-14+5.35210759e-15j, -3.89004695e-17+2.76226612e-15j,
        -3.67374563e-15-8.08479995e-15j],
       [-8.68842742e-15+1.79566577e-14j,  7.35454883e-15-1.98773856e-14j,
        -1.23309480e-15-1.64208581e-16j, ...,
         1.08437649e-15-1.42661373e-14j, -4.21884749e-15-2.73137715e-15j,
         3.26839427e-15+7.26824185e-15j],
       ...,
       [ 9.00000000e+00-2.13786998e-15j, -7.31413323e-15+1.32191195e-14j,
         3.33252339e-15+6.34271632e-15j, ...,
         1.21609612e-15+3.71609140e-15j, -6.12690643e-15-9.99436479e-15j,
        -3.71469862e-15-1.22542494e-14j],
       [-3.67351837e-15-1.82573956e-14j,  2.

From the above results, we can see that our vectors are very sparse if we take the IDFT

## Setting up the dataset for Pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

X = dense_data.T

X_tensor = torch.tensor(X,dtype=torch.cfloat)

dataset = TensorDataset(X_tensor,X_tensor)

dataloader = DataLoader(dataset,batch_size = 100,shuffle = True)

(1000, 100)


## Setting up the neural network

In [24]:
class ExponentialComplexLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(ExponentialComplexLinear, self).__init__()

        # Trainable phase angles (real-valued), this is where we are training the q-values in the context of the exercise
        self.phases = nn.Parameter(torch.randn(out_features, in_features) * 0.1)  # Small random initialization

    def forward(self, x):
        # Compute our matrix
        W = torch.exp(1j * self.phases)  # Enforces |W| = 1

        # Complex matrix multiplication
        return torch.matmul(x, W.t())

class ComplexDecoder(nn.Module):
    def __init__(self, encoding_dim, output_dim):
        super(ComplexDecoder, self).__init__()

        self.layers = nn.Sequential(
            ExponentialComplexLinear(encoding_dim, 128),  # Complex Linear Layer
            ComplexReLU(),  # Activation
            ExponentialComplexLinear(128, 256),
            ComplexReLU(),
            ExponentialComplexLinear(256, output_dim)  # Final output layer
        )

    def forward(self, x):
        return self.layers(x)

class ComplexReLU(nn.Module):
    def forward(self, x):
        return torch.complex(F.relu(x.real), F.relu(x.imag))  # Apply ReLU separately

class LearnedAutoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(LearnedAutoencoder, self).__init__()
        # Encoder: Maps from input_dim to a lower-dimensional complex encoding space.
        self.encoder = ExponentialComplexLinear(input_dim, encoding_dim)
        
        # Decoder: Maps from the encoding space back to the original input dimension.
        self.decoder = ComplexDecoder(encoding_dim, input_dim)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded



## Training the model

In [28]:
# Define the size of our "measurement" vector as encoding_dim. This needs to be larger than the sparsity of our matrix

encoding_dim = max_sparsity + 1

# Initialize model
model = LearnedAutoencoder(vector_size, encoding_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

def complex_mse_loss(input, target):
    return F.mse_loss(input.real, target.real) + F.mse_loss(input.imag, target.imag)

# Training loop
for epoch in range(1000):
    for batch in dataloader:
        inputs, targets = batch  # Unpack the tuple
        optimizer.zero_grad()
        output = model(inputs)
        loss = complex_mse_loss(output, targets)  # Custom loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.6f}")

Epoch 1, Loss: 4205088079872.000000
Epoch 2, Loss: 4305087488.000000
Epoch 3, Loss: 3548248539136.000000
Epoch 4, Loss: 5219619962880.000000
Epoch 5, Loss: 3042452701184.000000
Epoch 6, Loss: 2623023874048.000000
Epoch 7, Loss: 5904337469440.000000
Epoch 8, Loss: 2220238569472.000000
Epoch 9, Loss: 4720451125248.000000
Epoch 10, Loss: 4465557504000.000000
Epoch 11, Loss: 40096727040.000000
Epoch 12, Loss: 2155561222144.000000
Epoch 13, Loss: 1558163619840.000000
Epoch 14, Loss: 2909429301248.000000
Epoch 15, Loss: 2737407000576.000000
Epoch 16, Loss: 6558643126272.000000
Epoch 17, Loss: 2890331324416.000000
Epoch 18, Loss: 4591495151616.000000
Epoch 19, Loss: 5949719314432.000000
Epoch 20, Loss: 2520901484544.000000
Epoch 21, Loss: 2365512744960.000000
Epoch 22, Loss: 1380905385984.000000
Epoch 23, Loss: 2240828932096.000000
Epoch 24, Loss: 3343725625344.000000
Epoch 25, Loss: 2209521074176.000000
Epoch 26, Loss: 948561903616.000000
Epoch 27, Loss: 5741143916544.000000
Epoch 28, Loss: 

KeyboardInterrupt: 